In [1]:
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfTransformer
import json
import pickle
import numpy as np

In [3]:
#df = pd.read_table('../../dataset/Export_agg_country_prod_yr.txt', encoding="latin-1",sep=',',dtype={'year':int,'HS':str,'SITC':str, 'CUCI': str})
df = pd.read_table('../../dataset/Export_World_directo.csv', encoding="latin-1",sep=',',dtype={'year':int,'HS':str,'SITC':str, 'CUCI': str})


In [3]:
df = df[['year','flow','reporter','rep_iso','SITC','value']] #saco las columnas d ecadenas, subcadenas, flores, etc. 

df = df.drop_duplicates().reset_index() #elimino los duplicados (porque cadenas duplicaba filas)


In [3]:
#voy a reutilizar los códigos en el shiny
#pd.DataFrame(df.groupby(["reporter","rep_iso"]).size()).reset_index()[["reporter","rep_iso"]].to_csv("codigos_paises.csv",index=None)

In [12]:
class Procesamiento_LDA():
    
        def __init__(self, df, level=4):
            df = self.clean(df, level)
            productos = df.pivot_table(index=['rep_iso','year'],columns='SITC',values='value').columns
            product_df = pd.read_excel("../names/UN Comtrade Commodity Classifications.xlsx")
            ClassS3 = product_df[product_df["Classification"]=='S3']
            Codes = ClassS3.set_index('Code')['Description'].to_dict()
            self.df = df
            self.productos = productos
            self.Codes = Codes
        
        def clean(self, df, level):
            df = df[df["SITC"].notnull()]
            df["value"] = df["value"].fillna(0)
            df["level"]=df.apply(lambda x: len(x["SITC"]),1)
            df = df.loc[df["level"]==5, ['year', 'reporter', 'rep_iso','SITC','value']]
            df['SITC'] = df['SITC'].apply(lambda x: x[0:level]) #Me quedo con los primeros 4 dígitos
            df= df.groupby(['year','reporter','rep_iso','SITC']).sum()
            return df
        
        def matrix_transform(self, df):
            matrix = df.pivot_table(index=['rep_iso','year'],columns='SITC',values='value')
            matrix = matrix.fillna(0)
            matrix = matrix.to_sparse(fill_value=0)
            return matrix
        
        def lda_transform(self,k,save_model=True):
            #tfidf = TfidfTransformer() ### Anulo el TF-IDF
            matrix = self.matrix_transform(self.df)
            #matrix = tfidf.fit_transform(matrix)
            #### Corre en 25 cores porque es para ejecutar en server ###
            lda_model = LatentDirichletAllocation(n_components=k, max_iter=75, random_state=1234,batch_size=200, learning_method='batch', n_jobs=35) #instance 
            #### Corre en 25 cores porque es para ejecutar en server ###
            lda_model.fit(matrix) #fit
            if save_model:
                pickle.dump(lda_model, open("lda{}.pkl".format(k), 'wb')) #save    
            return lda_model, matrix, k
        
        def norm_comp(self,lda_model):
            norm_components = lda_model.components_ / lda_model.components_.sum(axis=1)[:, np.newaxis]
            return norm_components
        
        def print_components(self, lda_model):
            
            norm_components = self.norm_comp(lda_model)
            for i in range(lda_model.components_.shape[0]):
                topic = norm_components[i]
                print("Componente %d:" % (i+1))
                top_n = [self.productos[j] for j in np.argsort(topic)[:-15 :-1]]
                top_n_props = topic[np.argsort(topic)][:-15 :-1]
                for j,producto in enumerate(top_n):
                    try:
                        print("{}-{}. Prop: {}%".format(producto, self.Codes[producto],+np.round(top_n_props[j]*100,3)))
                    except:
                        pass
                print("----------------------")

        def save_results(self, lda_model,matrix, k):
            
            norm_components = self.norm_comp(lda_model)
            components_df = pd.DataFrame(norm_components)
            components_df.columns = self.productos

            components_df.to_csv("results/Dist_cadenas{}.csv".format(k), index=False)

            result = lda_model.transform(matrix)
            result = pd.DataFrame(result)
            result.index = df.pivot_table(index=['rep_iso','year'],columns='SITC',values='value').index
            result = result.reset_index()
            result.to_csv("results/Dist_paises{}.csv".format(k),index=False)
        



In [13]:
procesadora_lda = Procesamiento_LDA(df, 4)

In [14]:
# 2 componentes
lda_model, matrix, k = procesadora_lda.lda_transform(2,save_model = True)
procesadora_lda.save_results(lda_model,matrix, k)

# 4 componentes
lda_model, matrix, k = procesadora_lda.lda_transform(4,save_model = True)
procesadora_lda.save_results(lda_model,matrix, k)

# 6 componentes
lda_model, matrix, k = procesadora_lda.lda_transform(6,save_model = True)
procesadora_lda.save_results(lda_model,matrix, k)

# 8 componentes
lda_model, matrix, k = procesadora_lda.lda_transform(8,save_model = True)
procesadora_lda.save_results(lda_model,matrix, k)

# 10 componentes
lda_model, matrix, k = procesadora_lda.lda_transform(10,save_model = True)
procesadora_lda.save_results(lda_model,matrix, k)

# 20 componentes
lda_model, matrix, k = procesadora_lda.lda_transform(20,save_model = True)
procesadora_lda.save_results(lda_model,matrix, k)

# 30 componentes
lda_model, matrix, k = procesadora_lda.lda_transform(30,save_model = True)
procesadora_lda.save_results(lda_model,matrix, k)

# 40 componentes
lda_model, matrix, k = procesadora_lda.lda_transform(40,save_model = True)
procesadora_lda.save_results(lda_model,matrix, k)

# 50 componentes
lda_model, matrix, k = procesadora_lda.lda_transform(50,save_model = True)
procesadora_lda.save_results(lda_model,matrix, k)

# 100 componentes
lda_model, matrix, k = procesadora_lda.lda_transform(100,save_model = True)
procesadora_lda.save_results(lda_model,matrix, k)

# 200 componentes
lda_model, matrix, k = procesadora_lda.lda_transform(200,save_model = True)
procesadora_lda.save_results(lda_model,matrix, k)

In [11]:
matrix

value
year reporter rep_iso SITC              
1996 Albania  ALB     1110  6.278000e+03
                      1124  3.543100e+04
                      1211  9.888060e+05
                      1212  2.869330e+05
                      1223  7.593056e+06
                      2111  9.825830e+05
                      2112  2.468651e+06
                      2114  2.256390e+05
                      2116  4.175540e+05
                      2117  1.452140e+05
                      2119  3.062000e+04
                      2122  2.020400e+04
                      2237  4.208900e+04
                      2312  4.027200e+04
                      2322  1.181200e+04
                      2450  3.650870e+05
                      2482  6.851500e+04
                      2484  1.164588e+06
                      2485  9.926080e+05
                      2511  9.500000e+01
                      2633  2.209000e+03
                      2649  7.150000e+02
                      2665  4.903000e+03
                      2681  6.199900e+04
                      2682  1.881800e+04
                      2690  1.299960e+05
                      2721  2.748000e+03
                      2731  1.896000e+04
                      2732  1.880000e+02
                      2734  4.000000e+00
...                                  ...
2018 Zimbabwe ZWE     8924  3.000000e+01
                      8928  8.297500e+05
                      8931  4.726892e+06
                      8932  1.790500e+04
                      8933  1.280350e+05
                      8939  2.089250e+05
                      8942  4.976000e+03
                      8943  9.055360e+05
                      8947  1.861200e+04
                      8951  6.943000e+03
                      8952  2.962400e+05
                      8959  6.950000e+02
                      8961  1.243270e+05
                      8963  2.253457e+06
                      8965  8.995938e+06
                      8972  3.727000e+03
                      8973  6.867281e+07
                      8981  1.800000e+03
                      8982  1.500000e+02
                      8985  2.631000e+04
                      8987  1.610000e+02
                      8989  6.500000e+02
                      8991  2.192000e+03
                      8992  1.400000e+01
                      8993  4.750000e+02
                      8994  2.070000e+02
                      8996  7.758900e+04
                      8997  6.785500e+04
                      8999  1.012576e+06
                      9710  1.141529e+09

[1961767 rows x 1 columns]